# Understanding the Paillier Cryptosystem

2023, Thomas Liebig
[Compare my previous notes (German)](https://www-ai.cs.tu-dortmund.de/LEHRE/FACHPROJEKT/WS1516/download/paillier.pdf)

## Key Generation

In [1]:
# chose two primes
p=3
q=5

In [2]:
import math
import random

n=p*q 
phi=(p-1)*(q-1) # Eulers Phi
lambda1=math.lcm(p-1,q-1) # Carmichael number
g=random.randrange(n ** 2)
g_s=g
while True:
    g=(g+1) % (n ** 2)    # pick g randomly, note this is not i.i.d but allows us to test whether we tried all 
    if g==g_s:            # we tested all elements of the ring n^2
        raise ValueError('g can not not be found for p and q')
    if (math.gcd(g,n ** 2)==1) and (math.gcd(int((pow(g, lambda1) % (n ** 2) -1)/n),n)==1):
        # g is co-prime to n^2, and
        # L(g^lambda1) is co-prime to n , with L(x):=(x-1)/n (compare Decryption below)
        break


## Encryption
$$
\begin{aligned}
\mathcal{E}_g: \mathbb{Z}_n \times \mathbb{Z}_n^* & \rightarrow \mathbb{Z}_{n^2}^* \\
\mathcal{E}_g(x, y) & \rightarrow g^x \cdot y^n \bmod n^2 \\
\mathcal{E}_g(m, r) & \rightarrow g^m \cdot r^n \bmod n^2
\end{aligned}
$$

## Decryption
$$m=L(c^\lambda mod n^2)* L^{-1}(g^\lambda \text{mod} n^2)$$
mit $L(x)=(x-1)/n$

In [3]:
import numpy as np
from math import gcd

A = np.zeros((phi, n))

j = 1

names=[]

for i in range(phi):
    while gcd(j, n) != 1:
        j += 1
    names.append(j)
    for m in range(n):
        A[i, m] = (pow(g, m, pow(n, 2)) * pow(j, n, pow(n, 2))) % pow(n, 2)
    j += 1


In [4]:
def printMatrix(s,names):

    # Do heading
    print("     ", end="")
    for j in range(len(s[0])):
        print("%5d " % j, end="")
    print()
    print("     ", end="")
    for j in range(len(s[0])):
        print("------", end="")
    print()
    # Matrix contents
    for i in range(len(s)):
        print("%3d |" % (names[i]), end="") # Row nums
        for j in range(len(s[0])):
            print("%5d " % (s[i][j]), end="")
        print()  


### Inspect matrix of all ciphertexts A

In [5]:
printMatrix(A,names)

         0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
     ------------------------------------------------------------------------------------------
  1 |    1    29   166    89   106   149    46   209   211    44   151   104    91   164    31 
  2 |  143    97   113   127    83   157    53   187    23   217   218    22   188    52   158 
  4 |  199   146   184   161   169   176   154   191   139   206   124   221   109    11    94 
  7 |  118    47    13   152   133    32    28   137   148    17    43   122   163     2    58 
  8 |  107   178   212    73    92   193   197    88    77   208   182   103    62   223   167 
 11 |   26    79    41    64    56    49    71    34    86    19   101     4   116   214   131 
 13 |   82   128   112    98   142    68   172    38   202     8     7   203    37   173    67 
 14 |  224   196    59   136   119    76   179    16    14   181    74   121   134    61   194 


Observe the homomorphic property: Take two numbers from two columns $a$ and $b$, multiply these number and observe that their product is in the column $a+b$.

### Inspect Decryption Steps I: Inspect matrix $A^\lambda$

In [6]:
A_l=np.empty(A.shape)
for i in range(0,A.shape[0]):
        for j in range(0,A.shape[1]):
            A_l[i][j]=pow(int(A[i,j]),lambda1,pow(n, 2))
printMatrix(A_l,names)

         0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
     ------------------------------------------------------------------------------------------
  1 |    1   106   211    91   196    76   181    61   166    46   151    31   136    16   121 
  2 |    1   106   211    91   196    76   181    61   166    46   151    31   136    16   121 
  4 |    1   106   211    91   196    76   181    61   166    46   151    31   136    16   121 
  7 |    1   106   211    91   196    76   181    61   166    46   151    31   136    16   121 
  8 |    1   106   211    91   196    76   181    61   166    46   151    31   136    16   121 
 11 |    1   106   211    91   196    76   181    61   166    46   151    31   136    16   121 
 13 |    1   106   211    91   196    76   181    61   166    46   151    31   136    16   121 
 14 |    1   106   211    91   196    76   181    61   166    46   151    31   136    16   121 


### Inspect Decryption Steps II: Inspect matrix $L(A^\lambda)\text{ mod }n^2$

In [7]:
A_l=np.empty(A.shape)
for i in range(0,A.shape[0]):
        for j in range(0,A.shape[1]):
            A_l[i][j]=(pow(int(A[i,j]),lambda1,pow(n, 2))-1)/n
printMatrix(A_l,names)

         0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
     ------------------------------------------------------------------------------------------
  1 |    0     7    14     6    13     5    12     4    11     3    10     2     9     1     8 
  2 |    0     7    14     6    13     5    12     4    11     3    10     2     9     1     8 
  4 |    0     7    14     6    13     5    12     4    11     3    10     2     9     1     8 
  7 |    0     7    14     6    13     5    12     4    11     3    10     2     9     1     8 
  8 |    0     7    14     6    13     5    12     4    11     3    10     2     9     1     8 
 11 |    0     7    14     6    13     5    12     4    11     3    10     2     9     1     8 
 13 |    0     7    14     6    13     5    12     4    11     3    10     2     9     1     8 
 14 |    0     7    14     6    13     5    12     4    11     3    10     2     9     1     8 


In [8]:
def gcdExtended(a, b): 
    if a == 0 :  
       return b,0,1
             
    gcd,x1,y1 = gcdExtended(b % a, a) 
     
    x = y1 - (b//a) * x1 
    y = x1 
     
    return gcd,x,y


### Inspect Decryption Steps III: Inspect matrix $L(A^\lambda)/L(g^\lambda)\text{ mod }n$

In [9]:
Lg = (pow(g,lambda1,pow(n,2))-1)/n
Lg_inv = (gcdExtended(Lg,n)[1]+n) % n

#((modpower(A,lambda,n^2)-1)/n) * Lg_inv) %% n

A_i=np.empty(A.shape)
for i in range(0,A.shape[0]):
        for j in range(0,A.shape[1]):
            A_i[i][j]=(((pow(int(A[i,j]),lambda1,pow(n, 2))-1)/n) * Lg_inv ) % n
printMatrix(A_i,names)

         0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
     ------------------------------------------------------------------------------------------
  1 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
  2 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
  4 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
  7 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
  8 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
 11 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
 13 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
 14 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
